In [2]:
!pip install datasets scikit-learn pandas sagemaker boto3 pyyaml

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached sagemaker-2.232.3-py3-none-any.whl.metadata (16 kB)
  Using cached boto3-1.35.53-py3-none-any.whl.metadata (6.7 kB)
  Using cached pyarrow-18.0.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.10.10-cp311-cp311-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached pat

In [3]:
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFace
import yaml
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import Session
import json
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.xgboost.model import XGBoostModel
from sagemaker.inputs import TrainingInput

/Users/kevinkuc/Documents/real_estate_analysis/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/kevinkuc/Library/Application Support/sagemaker/config.yaml


In [42]:
# Step 6: Download a file from your S3 bucket
s3_client = boto3.client('s3')

# Step 3: Define your S3 bucket name
bucket_name = 'sagemaker-us-east-1-025066244931'
def download_file_from_s3(s3_file, local_file):
    try:
        s3_client.download_file(bucket_name, s3_file, local_file)
        print(f"Successfully downloaded {s3_file} from {bucket_name} to {local_file}")
    except Exception as e:
        print(f"Error downloading file: {e}")

# Example: Download a file from S3
s3_file_to_download = 'CC_baseline_no_header.csv'  # Replace with the name of your file in S3
local_file_to_save = 'CC_baseline_no_header.csv'  # Replace with the desired local file name
download_file_from_s3(s3_file_to_download, local_file_to_save)

Successfully downloaded validation.csv from sagemaker-us-east-1-025066244931 to validation.csv


In [43]:
s3_file_to_download = './baseline/CC_baseline_no_header.csv'  # Replace with the name of your file in S3
local_file_to_save = './baseline/CC_baseline_no_header.csv'  # Replace with the desired local file name
download_file_from_s3(s3_file_to_download, local_file_to_save)

Successfully downloaded train.csv from sagemaker-us-east-1-025066244931 to train.csv


In [14]:
data_path = "./baseline/CC_baseline_no_header.csv"
df = pd.read_csv(data_path, skiprows=1, header=None)

df

,0,1,2,3,4,5,6,7,8,9,...,208,209,210,211,212,213,214,215,216,217
0,658,1850,2,2.5,5,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,467,1469,2,1.0,38,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,565,2945,2,2.5,4,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,56,2536,3,2.5,3,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,310,2687,3,2.5,3,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10793,6,877,2,1.0,46,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10794,105,1236,2,2.0,2,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10795,105,1331,2,2.0,2,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10796,105,1705,2,2.0,2,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Training the model (Creates training and validation data)

In [ ]:
s3 = boto3.client('s3')
bucket_name = 'sagemaker-us-east-1-025066244931'  # Replace with your S3 bucket name

In [84]:
file_path = "./baseline"

# Extract features and labels
X = df.iloc[:,:1]
y = df.iloc[:,1:]

with open("./config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, **config["train_test_split"])

# Save the train and test sets as CSV files
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.to_csv(f'{file_path}/train.csv', index=False)
test_data.to_csv(f'{file_path}/validation.csv', index=False)

# Upload them to S3
s3.upload_file(f'{file_path}/train.csv', bucket_name, f'{file_path}/train.csv')
s3.upload_file(f'{file_path}/validation.csv', bucket_name, f'{file_path}/validation.csv')


In [85]:
with open("./config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)

# set an output path where the trained model will be saved
bucket = sagemaker.Session().default_bucket()
prefix = 'DEMO-xgboost-as-a-built-in-algo'

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=config['hyperparameters'],
                                          role=sagemaker.get_execution_role(),
                                          instance_count=config['sagemaker_config']['instance_count'], 
                                          instance_type=config['sagemaker_config']['instance_type'], 
                                          volume_size=config['sagemaker_config']['volume_size'],
                                          output_path=config['sagemaker_config']['output_path'])

content_type = "csv"
# define the data type and paths to the training and validation datasets
train_input = TrainingInput(
    s3_data=f's3://{bucket_name}/train',  # S3 path for training data
    content_type='csv'
)
validation_input = TrainingInput(
    s3_data=f's3://{bucket_name}/validation',  # S3 path for validation data
    content_type='csv'
)

# execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-10-25-03-39-12-308


2024-10-25 03:39:13 Starting - Starting the training job...
2024-10-25 03:39:27 Starting - Preparing the instances for training...
2024-10-25 03:40:06 Downloading - Downloading the training image...
2024-10-25 03:40:46 Training - Training image download completed. Training in progress....
2024-10-25 03:41:12 Uploading - Uploading generated training model[2024-10-25 03:41:04.479 ip-10-0-81-47.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-10-25 03:41:04.506 ip-10-0-81-47.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-10-25:03:41:04:INFO] Imported framework sagemaker_xgboost_container.training
[2024-10-25:03:41:04:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
[2024-10-25:03:41:04:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-25:03:41:04:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-10-25:03:41:04:INFO] Determined 0 GPU(s) available on the in

## Endpoint

In [86]:
model_artifact = f's3://{bucket_name}/output/sagemaker-xgboost-2024-10-25-03-39-12-308/output/model.tar.gz'

xgb_model = XGBoostModel(
    model_data=model_artifact,  # Path to the trained model artifact from the training job
    role='real_estate_endpoint',
    entry_point=None,           # Not needed for built-in XGBoost
    framework_version='1.7-1',  # Version of XGBoost used in training (align with image_uri)
)

# Configure serverless inference
serverless_config = sagemaker.serverless.ServerlessInferenceConfig(
    memory_size_in_mb=2048,  # Memory allocated for requests
    max_concurrency=5         # Maximum concurrent invocations
)

# Deploy the model using serverless inference
predictor = xgb_model.deploy(
    serverless_inference_config=serverless_config,
    container_startup_health_check_timeout=600  # Increase timeout to 300 seconds
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-10-25-03-48-16-520
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-10-25-03-48-17-285
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-10-25-03-48-17-285


-----------------------------------------!

## Test Endpoint

In [88]:
# Initialize a SageMaker session
sagemaker_session = Session()

# Replace with your endpoint name
endpoint_name = 'sagemaker-xgboost-2024-10-25-03-48-17-285'

# Create a Predictor object
predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)

# Set the serializer for the predictor to CSV
predictor.serializer = CSVSerializer()

# Prepare the input data as CSV
csv_input = "8523,384444"

# Make a prediction using CSV input
response = predictor.predict(
    data=csv_input,  # Send the CSV data
    initial_args={'ContentType': 'text/csv'}  # Specify the content type as CSV
)

# Read and decode the response
result = response.decode('utf-8')
print("Prediction result:", result)

Prediction result: 4.0

